In [20]:
from IPython.display import display
import pandas as pd
import requests
import re
from urllib.request import urlopen
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
#from fa2 import ForceAtlas2
from scipy import stats
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import io
from tqdm import tqdm
from heapq import nlargest 
import seaborn as sns
import simplemma

sns.set()

In [21]:
giant_df = pd.read_csv("Data/Giant.csv")
display(giant_df.head())

,Party,Person,Tweets
0,socialdemokratiet,magnus_heunicke,"['promises made promises kept', 'covidsmitten ..."
1,socialdemokratiet,nicolai_wammen,['vej spændende aften gode kollegaer god valga...
2,socialdemokratiet,mattias_tesfaye,"['nytår ps vandt', 'liekaas fremragende filmen..."
3,venstre,jakob_ellemann,['fesagen skadet danmarks efterretningstjenest...
4,venstre,soren_gade,"['snothvalpen ved grimmebriller', 'sagt lige s..."


In [22]:
import ast
# Convert string to list and join strings
# t = ast.literal_eval(giant_df.iloc[0,2]) # skal måske benyttes 
# t = ' '.join(t)

# Tokenize and clean tweets
wnl = nltk.WordNetLemmatizer() 
STOPWORDS = nltk.corpus.stopwords.words('danish') + ["http"]

def clean_tweet(tweet):
    tokensraw = word_tokenize(tweet) #get tokens 
    tokens = [word.lower() for word in tokensraw if word.isalnum()] #only get words and numbers
    tokens = [simplemma.lemmatize(t, lang='da') for t in tokens]
    tokens = [w for w in tokens if w.lower() not in STOPWORDS] #remove stopwords
    return tokens

In [23]:
giant_df['CT'] = giant_df['Tweets']
for w in range(len(giant_df)):
    giant_df['CT'][w] = ' '.join(giant_df['CT'][w])
    giant_df['CT'][w] = clean_tweet(giant_df['Tweets'][w])
   

In [24]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [25]:
giant_df.CT[1]

['spænde',
 'aften',
 'gode',
 'kollega',
 'god',
 'valgaften',
 'usikker',
 'tid',
 'grund',
 'glæde',
 'dag',
 'lave',
 'ledighedstal',
 'fald',
 'person',
 'ledighed',
 'pct',
 'vigtig',
 'fortsætte',
 'sikker',
 'hånd',
 'rat',
 'passe',
 'godte',
 'dansk',
 'job',
 'virksomhed',
 'dkpol',
 'poul',
 'helle',
 'anbefale',
 'østjyderne',
 'stemme',
 'personlig',
 'tirsdag',
 'flotte',
 'dejlig',
 'skulderklap',
 'to',
 'tidlig',
 'statsminister',
 'almindelig',
 'familie',
 'stå',
 'miste',
 'tusindvis',
 'kr',
 'mens',
 'direktør',
 'forgylde',
 'moderaternes',
 'skatteplan',
 'moderat',
 'voldsom',
 'løkke',
 'tone',
 'ren',
 'flag',
 'føre',
 'blå',
 'borgerlig',
 'økonomisk',
 'politik',
 'forslag',
 'akutpakke',
 'bombe',
 'kommune',
 'økonomi',
 'v',
 'finde',
 'mia',
 'kr',
 'ved',
 'konkurrenceudsættelse',
 'tro',
 'v',
 'sende',
 'masse',
 'udbud',
 'så',
 'falde',
 'imod',
 'mia',
 'kr',
 'himmel',
 'jan',
 'få',
 'stor',
 'konsekvens',
 'nære',
 'velfærd',
 'mentor',
 'pol

In [26]:
sentences = [giant_df.CT[0],giant_df.CT[6]]
# sentences = [sent.lower().split(" ") for sent in sentences]
jaccard_similarity(sentences[0], sentences[1])

0.2572194561470701

# foreslag på fremgfangsmåde

In [27]:
# converting a string of text into a vector. Using teh transformer BERT model

# Step one: Use BERT to convert our text into a vector
# Step two:Get the cosine similarity (the cosine of the angle between the two vectors) 
    # of a fixed twitter profiles (vector) and all the other ones
# Step three: Pick the twitter profiles (vectors) with the largest cosine similarity.




In [28]:
# step one
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

text_data = giant_df.CT
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
X = np.array(giant_df.CT)
# cos_sim_data = pd.DataFrame(cosine_similarity(X))

In [37]:
# https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4

def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_parties =False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:10]
  party_recomm =  giant_df['Party'].loc[index_recomm].values
  result = {'PArty':party_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('The watched movie is this one: %s \n'%(giant_df['Person'].loc[index]))
    k=1
    for movie in party_recomm:
      print('The number %i recommended movie is this one: %s \n'%(k,movie))
  if print_recommendation_plots==True:
    print('The plot of the watched movie is this one:\n %s \n'%(giant_df['CT'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Overview'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  if print_parties==True:
    print('The party of the twitter profile is this one:\n %s \n'%(giant_df['Party'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Party'].loc[index_recomm[q]]
      print('The plot of the number %i recommended party is this one:\n %s \n'%(k,plot_q))
      k=k+1
  return result



In [59]:
# check if 'sundhedsminister' is in the list

if 'sundhedsministeren' in doc[0]:
    print('yes')